In [ ]:
# 크레온 api를 사용해서 데이터 수집
import win32com.client
import pandas as pd

class Creon:
    def __init__(self):
        self.obj_CpCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr') # 주식 코드와 관련된 여러 기능을 제공
        self.obj_CpCybos = win32com.client.Dispatch('CpUtil.CpCybos') # 크레온 플러스의 연결 상태를 확인
        self.obj_StockChart = win32com.client.Dispatch('CpSysDib.StockChart') # 주식 차트 데이터를 요청하는 기능을 제공

    def creon_7400_주식차트조회(self, code, date_from, date_to):
        # 크레온 플러스와의 연결 상태를 확인합니다. 연결되지 않은 경우 함수를 종료
        b_connected = self.obj_CpCybos.IsConnect
        if b_connected == 0:
            print("연결 실패")
            return None

        # 차트 데이터로부터 조회할 필드를 정의합니다. 여기서는 날짜, 시간, 시가, 고가, 저가, 종가, 거래량을 의미
        list_field_key = [0, 1, 2, 3, 4, 5, 8]
        list_field_name = ['date', 'time', 'open', 'high', 'low', 'close', 'volume']
        # list_field_name 리스트에 있는 각 항목 이름으로 키를 갖고, 빈 리스트를 값으로 하는 딕셔너리 dict_chart를 생성하는 코드
        dict_chart = {name: [] for name in list_field_name}
        # """
        # {
        #     'date': [],
        #     'time': [],
        #     'open': [],
        #     'high': [],
        #     'low': [],
        #     'close': [],
        #     'volume': []
        # }
        # """

        # 크레온 API를 사용하여 주식 차트 데이터를 요청
        # 차트 데이터 요청을 위한 조건을 설정합니다. 주식 코드, 조회 기간, 조회할 필드 등을 지정
        self.obj_StockChart.SetInputValue(0, 'A'+code) # 주식 코드 앞에 'A'를 붙여서 전체 주식 코드
        self.obj_StockChart.SetInputValue(1, ord('1'))  # 0: 개수, 1: 기간
        self.obj_StockChart.SetInputValue(2, date_to)  # 조회할 기간의 종료일
        self.obj_StockChart.SetInputValue(3, date_from)  # 조회할 기간의 시작일
        self.obj_StockChart.SetInputValue(5, list_field_key)  # list_field_key는 조회할 차트 데이터의 필드 인덱스(예: 날짜, 시가, 종가 등)를 리스트 형태로 포함
        self.obj_StockChart.SetInputValue(6, ord('D'))  # 'D'(일), 'W'(주), 'M'(월), 'm'(분), 'T'(틱) 차트의 시간 단위를 설정
        # 설정한 조건에 따라 차트 데이터를 요청
        self.obj_StockChart.BlockRequest()

        # 데이터 요청의 결과 상태와 메시지를 확인합니다. 오류가 있는 경우 함수를 종료
        status = self.obj_StockChart.GetDibStatus()
        msg = self.obj_StockChart.GetDibMsg1()
        print("통신상태: {} {}".format(status, msg))
        if status != 0:
            return None

        # 요청한 데이터의 총 개수를 가져옵니다.
        cnt = self.obj_StockChart.GetHeaderValue(3)  # 수신개수
        for i in range(cnt):
            # 수신된 차트 데이터를 순회하며, 각 필드별로 값을 추출하여 사전(dict_chart)에 저장
            dict_item = (
                # 이 구문은 각 필드명(name)을 키로, 해당 필드의 데이터 값을(self.obj_StockChart.GetDataValue(pos, i)) 값으로 하는 딕셔너리를 생성
                {name: self.obj_StockChart.GetDataValue(pos, i) # pos는 필드의 위치(인덱스), i는 현재 가져오려는 데이터의 인덱스
                for pos, name in zip(range(len(list_field_name)), list_field_name)}
            )
            for k, v in dict_item.items():
                dict_chart[k].append(v)

        print("차트: {} {}".format(cnt, dict_chart))
        return pd.DataFrame(dict_chart, columns=list_field_name)


creon = Creon()
print(creon.creon_7400_주식차트조회('035420', 20150101, 20171231))